In [1]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "all_hourly_weather_events_v2.csv"

OUTPUT_DATA_FILE = "all_features"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [4]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [5]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [6]:
# Technicaly it's feature engineering

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories
le = preprocessing.LabelEncoder()
df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0)

print(df_weather_events.dtypes)
df_weather_events.sample(10)

city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspeed           

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
158214,"Львів, Україна",2022-07-31,21.6,16.8,19.4,17.3,87.7,6.200,8.33,111.5,9.6,3.0,21129.0,76058.0,0.07,8.0,19.6,88.79,17.7,0.0,0.0,0.0,0.0,5.0,18.4,8.3,186.5,1009.0,24.1,100.0,12.0,0.0,0.0,10.0,3.0,13.0,NaN,NaN,NaN,0.0,False
123956,"Рівне, Україна",2022-06-17,20.8,11.2,15.6,12.0,79.7,4.855,8.33,119.4,10.4,6.0,18124.0,77398.0,0.59,14.0,20.8,57.10,12.0,0.0,0.0,0.0,0.0,5.0,17.3,9.0,291.3,1016.0,24.1,94.8,466.0,1.7,5.0,10.0,3.0,17.0,NaN,NaN,NaN,0.0,False
33555,"Київ, Україна",2022-09-25,15.9,5.9,10.6,5.5,72.5,0.000,0.00,161.8,14.0,6.0,24505.0,67798.0,0.96,14.0,14.4,54.34,5.3,0.0,0.0,0.0,0.0,5.0,39.2,15.1,275.1,1015.0,24.1,83.9,586.0,2.1,6.0,10.0,4.0,10.0,NaN,NaN,NaN,0.0,False
40194,"Херсон, Україна",2022-07-15,31.9,18.4,25.4,15.5,60.2,0.000,0.00,337.4,29.2,9.0,18657.0,74378.0,0.53,18.0,29.9,39.36,14.6,0.0,0.0,0.0,0.0,5.0,28.8,19.1,270.3,1012.0,24.1,45.4,455.0,1.6,5.0,10.0,4.0,21.0,NaN,NaN,NaN,0.0,False
79880,"Харків, Україна",2022-03-13,-2.0,-11.0,-5.7,-10.8,68.6,0.000,0.00,127.8,11.1,4.0,21209.0,63394.0,0.34,18.0,-4.0,58.23,-11.0,0.0,0.0,0.0,7.0,5.0,11.5,3.6,50.0,1029.0,10.0,6.2,126.0,0.5,0.0,10.0,0.0,20.0,11.0,10.0,6.444444,0.0,True
110538,"Ужгород, Ужгородський район, Україна",2022-10-26,13.4,9.0,10.9,9.8,93.4,1.136,8.33,73.1,6.2,4.0,29249.0,66088.0,0.01,14.0,12.6,85.99,10.3,0.0,0.0,0.0,0.0,5.0,15.8,5.1,140.0,1023.7,30.3,77.5,420.0,1.5,4.0,10.0,4.0,7.0,NaN,NaN,NaN,0.0,False
76865,"Миколаїв, Україна",2022-10-16,15.8,4.9,9.9,2.8,62.4,0.000,0.00,153.3,13.3,6.0,25866.0,65008.0,0.68,7.0,5.0,77.53,1.4,0.0,0.0,0.0,0.0,5.0,16.6,8.3,25.5,1027.0,24.1,0.0,0.0,NaN,0.0,10.0,0.0,14.0,NaN,NaN,NaN,0.0,False
130639,"Житомир, Україна",2022-03-29,14.9,5.7,10.6,0.1,49.2,1.000,4.17,137.5,12.0,6.0,24526.0,70350.0,0.89,1.0,7.9,42.42,-4.1,0.0,0.0,0.0,0.0,5.0,34.6,18.7,248.8,1012.0,24.1,0.0,0.0,NaN,0.0,10.0,0.0,6.0,8.0,16.0,42.327778,0.0,True
171380,"Черкаси, Україна",2022-03-26,15.5,1.5,8.6,0.4,59.5,1.000,8.33,179.0,15.5,7.0,20529.0,65622.0,0.78,12.0,14.0,40.26,0.7,0.0,0.0,0.0,0.0,5.0,49.3,28.1,237.7,1016.0,24.1,54.1,664.0,2.4,7.0,10.0,4.0,23.0,NaN,NaN,NaN,0.0,False
36177,"Київ, Україна",2022-12-28,3.2,0.9,1.9,-1.2,80.3,0.000,0.00,29.4,2.6,2.0,28705.0,57633.0,0.15,21.0,1.2,82.80,-1.4,0.0,0.0,0.0,1.4,5.0,16.6,7.6,232.6,1024.0,24.1,100.0,0.0,NaN,0.0,10.0,3.0,10.0,NaN,NaN,NaN,0.0,False


### Calculate TF-IDF matrix

In [7]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 5828)


### Merge final dataset

In [8]:
# Merge tfidf matrix with weather events dataset (takes 1m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="left",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")
print(df_final.shape)
df_final.sample(5)

(195084, 5869)


city_resolvedAddress day_datetime  day_tempmax  day_tempmin  \
60389           Чернігів, Україна   2022-12-17          1.1          0.5   
170132  Івано-Франківськ, Україна   2023-01-08          1.8         -0.3   
16013      Кропивницький, Україна   2022-12-27          5.7         -0.3   
122286             Рівне, Україна   2022-04-09         15.6          4.6   
47709           Чернівці, Україна   2022-06-21         24.9         14.7   

        day_temp  day_dew  day_humidity  day_precip  day_precipcover  \
60389        0.9      0.7          98.5       8.500            33.33   
170132       0.7      0.3          97.2       0.000             0.00   
16013        3.2      2.1          93.1       6.000             4.17   
122286       8.5      4.5          76.8       5.576             8.33   
47709       19.8     11.8          60.8       0.000             0.00   

        day_solarradiation  day_solarenergy  day_uvindex  day_sunrise  \
60389                  5.1              0.4          0.0      28530.0   
170132                24.8              2.1          2.0      29619.0   
16013                  8.6              0.7          1.0      27747.0   
122286               134.2             11.7          6.0      23645.0   
47709                242.4             20.9          9.0      18921.0   

        day_sunset  day_moonphase  hour_datetime  hour_temp  hour_humidity  \
60389      56775.0           0.78            7.0        0.8         100.00   
170132     60126.0           0.53            9.0        0.2          96.44   
16013      57694.0           0.12           15.0        5.4          91.97   
122286     72011.0           0.26            8.0        7.4          81.26   
47709      76848.0           0.72           15.0       19.5          54.91   

        hour_dew  hour_precip  hour_precipprob  hour_snow  hour_snowdepth  \
60389        0.9          0.0              0.0        0.0             6.2   
170132      -0.3          0.0              0.0        0.0             0.0   
16013        4.2          0.0              0.0        0.0             0.2   
122286       4.4          0.0              0.0        0.0             0.0   
47709       10.2          0.0              0.0        0.0             0.0   

        hour_preciptype  hour_windgust  hour_windspeed  hour_winddir  \
60389               5.0           11.9             5.8         167.5   
170132              5.0           31.7            15.8         122.6   
16013               5.0           20.2             9.4         245.5   
122286              3.0           31.3            15.8         175.4   
47709               5.0           43.2            24.3         337.0   

        hour_pressure  hour_visibility  hour_cloudcover  hour_solarradiation  \
60389          1014.0              0.0            100.0                  0.0   
170132         1023.0             24.1            100.0                  7.0   
16013          1015.0             24.1             94.1                 63.0   
122286          996.0             13.0            100.0                 27.0   
47709          1007.5             15.8             76.3                884.0   

        hour_solarenergy  hour_uvindex  hour_severerisk  hour_conditions  \
60389                NaN           0.0             10.0              3.0   
170132               0.0           0.0             10.0              3.0   
16013                0.2           1.0             10.0              3.0   
122286               0.1           0.0             10.0              3.0   
47709                3.2           9.0             10.0              4.0   

        region_id  event_alarms_past_24  event_simultaneous_alarms  \
60389        25.0                   NaN                        NaN   
170132        9.0                   NaN                        NaN   
16013        11.0                   NaN                        NaN   
122286       17.0                   NaN                        NaN   
47709        24.0              

In [9]:
# Save final dataset
# with open(f"{PREPROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
#    pickle.dump(df_final, handle)